In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6"  # 指定使用 GPU 3

In [2]:
import topicfeaturesmall

2024-06-23 21:03:10.296408: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-23 21:03:10.335958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 21:03:10.335993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 21:03:10.337201: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 21:03:10.344741: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import numpy as np
from sklearn.model_selection import GroupKFold
import pandas as pd

PATH = "/home/mcq2/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
train = pd.read_csv(PATH + "train.csv")

# X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
# y = np.array([1, 2, 3, 4, 5, 6])
topic = topicfeaturesmall.predict_chunk(train)
#group = topic['topic']


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

/home/mcq2/anaconda3/envs/aes/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
group = topic['topic']

In [5]:
group

0        1
1        6
2        3
3        4
4        2
        ..
17302    4
17303    5
17304    0
17305    4
17306    4
Name: topic, Length: 17307, dtype: int64

In [14]:
X = train
y_split = train['score'].astype(int).values

In [23]:
group_kfold.get_n_splits(X, y_split, group)

7

In [26]:
GroupKFold(n_splits=14)
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split, group)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}, group={group[train_index]}")
    print(f"  Test:  index={test_index}, group={group[test_index]}")

Fold 0:
  Train: index=[    0     1     3 ... 17304 17305 17306], group=0        1
1        6
3        4
4        2
5        2
        ..
17302    4
17303    5
17304    0
17305    4
17306    4
Name: topic, Length: 13822, dtype: int64
  Test:  index=[    2    10    12 ... 17282 17286 17291], group=2        3
10       3
12       3
14       3
19       3
        ..
17271    3
17277    3
17282    3
17286    3
17291    3
Name: topic, Length: 3485, dtype: int64
Fold 1:
  Train: index=[    0     1     2 ... 17304 17305 17306], group=0        1
1        6
2        3
3        4
4        2
        ..
17300    0
17302    4
17304    0
17305    4
17306    4
Name: topic, Length: 14261, dtype: int64
  Test:  index=[    6    16    21 ... 17292 17301 17303], group=6        5
16       5
21       5
24       5
28       5
        ..
17284    5
17290    5
17292    5
17301    5
17303    5
Name: topic, Length: 3046, dtype: int64
Fold 2:
  Train: index=[    0     1     2 ... 17301 17303 17304], group=0        1

In [24]:
LOAD = True # re-train
# Define the number of splits for cross-validation
n_splits = 7
models = []

if not LOAD:
    for i in range(n_splits):
        models.append(lgb.Booster(model_file=f'kaggle/input/aes-lgbm/fold_{i+1}.txt'))
else:
    # Initialize StratifiedKFold with the specified number of splits
    #skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    group_kfold = GroupKFold(n_splits=n_splits)
    # Lists to store scores
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
    #callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
    # Loop through each fold of the cross-validation
    i=1
    for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split, group)):
        # Split the data into training and testing sets for this fold
        print('fold',i)
        # X_train_fold, X_test_fold = X[train_index], X[test_index]
        # y_train_fold, y_test_fold, y_test_fold_int = y_split[train_index], y_split[test_index], y_split[test_index]

        # model = lgb.LGBMRegressor(
        #             objective = qwk_obj,
        #             metrics = 'None',
        #             learning_rate = 0.05,
        #             max_depth = 5,
        #             num_leaves = 10,
        #             colsample_bytree=0.3,
        #             reg_alpha = 0.7,
        #             reg_lambda = 0.1,
        #             n_estimators=700,
        #             random_state=42,
        #             extra_trees=True,
        #             class_weight='balanced',
        #             device='gpu',
        #             verbosity = - 1)

        # # Fit the model on the training data for this fold  
        # predictor = model.fit(X_train_fold,
        #                       y_train_fold,
        #                       eval_names=['train', 'valid'],
        #                       eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
        #                       eval_metric=quadratic_weighted_kappa,
        #                       callbacks=callbacks
        #                      )

        # models.append(predictor)
        # # Make predictions on the test data for this fold
        # predictions_fold = predictor.predict(X_test_fold)
        # predictions_fold = predictions_fold + a
        # predictions_fold = predictions_fold.clip(1, 6).round()
        # predictions.append(predictions_fold)
        # # Calculate and store the F1 score for this fold
        # f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        # f1_scores.append(f1_fold)

        # # Calculate and store the Cohen's kappa score for this fold
        # kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        # kappa_scores.append(kappa_fold)
        # predictor.booster_.save_model(f'kaggle/out/aes-lgbm/fold_{i}.txt')

        # print(f'F1 score across fold: {f1_fold}')
        # print(f'Cohen kappa score across fold: {kappa_fold}')
        # i+=1

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6


In [25]:
fold 0

SyntaxError: invalid syntax (1611499881.py, line 1)